# Download landsat scenes over the San Juan Islands

This notebook demonstrates:
* Using the pystac client to retrieve images from microsofts cloud storage
* Saving those images to netcdf

Code adapted from: https://medium.com/@geonextgis/getting-started-with-microsoft-planetary-computer-stac-api-67cbebe96e5e

In [1]:
import pystac_client
import planetary_computer
import odc.stac
from pystac.extensions.eo import EOExtension as eo

/home/caprice/micromamba/envs/summerfog/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [19]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)

#opening a connection to the STAC API endpoint hosted by the Planetary Computer platform
#check website for more info


In [21]:
import pandas as pd
landsat = catalog.get_collection("landsat-c2-l2")

pd.DataFrame(landsat.summaries.get_list("eo:bands"))

,name,common_name,description,center_wavelength,full_width_half_max
0,TM_B1,blue,Visible blue (Thematic Mapper),0.49,0.07
1,TM_B2,green,Visible green (Thematic Mapper),0.56,0.08
2,TM_B3,red,Visible red (Thematic Mapper),0.66,0.06
3,TM_B4,nir08,Near infrared (Thematic Mapper),0.83,0.14
4,TM_B5,swir16,Short-wave infrared (Thematic Mapper),1.65,0.20
5,TM_B6,lwir,Long-wave infrared (Thematic Mapper),11.45,2.10
6,TM_B7,swir22,Short-wave infrared (Thematic Mapper),2.22,0.27
7,ETM_B1,blue,Visible blue (Enhanced Thematic Mapper Plus),0.48,0.07
8,ETM_B2,green,Visible green (Enhanced Thematic Mapper Plus),0.56,0.08
9,ETM_B3,red,Visible red (Enhanced Thematic Mapper Plus),0.66,0.06


In [9]:
# FHL location on San Juans
bbox_of_interest = [-123.33711789011167, 48.4, -122.65866282644907, 48.93242519875351]

#for times_of_interest, can do a list of dates, or a range of dates:


times_of_interest = [
    "2024-06-30/2024-06-30",
    "2024-07-08/2024-07-08",
    "2024-07-16/2024-07-16",
    "2024-07-24/2024-07-24",
    "2024-08-01/2024-08-01",
    "2024-08-09/2024-08-09",
    "2024-08-17/2024-08-17",
    "2024-08-25/2024-08-25",
    "2024-09-02/2024-09-02",
    "2024-09-10/2024-09-10",
    "2024-09-18/2024-09-18",
    "2024-09-26/2024-09-26",
    ]



# times_of_interest = [
#     "2022-07-27/2022-07-27",
#     "2022-08-04/2022-08-04",
#     "2022-08-12/2022-08-12",
#     "2022-08-20/2022-08-20",
#     "2022-09-29/2022-09-29",
#     "2021-06-30/2021-07-01",
# ]
#these are all the good dates from summer2022, listed in word doc from jessica

# Finding matching scenes for bbox_of_interest and times_of_interest

In [4]:
#for a range of times:
items_list = []
   #goes through times in list above
    # Query the catalog
search = catalog.search(
    collections=["landsat-c2-l2", "landsat-c1-l2"],
    bbox=bbox_of_interest,
        datetime=times_of_interest,
        # #  Uncomment the two lines below to sort the image collection based on 'cloud_cover'
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
)

items = search.item_collection()
print(f"Returned {len(items)} Items")
items_list.append(items)

Exception: too many datetime components (max=2, actual=12): ['2020-07-05/2020-07-05', '2020-07-13/2020-07-13', '2020-07-21/2020-07-21', '2020-07-29/2020-07-29', '2020-08-06/2020-08-06', '2020-08-14/2020-08-14', '2020-08-22/2020-08-22', '2020-08-30/2020-08-30', '2020-09-07/2020-09-07', '2020-09-15/2020-09-15', '2020-09-23/2020-09-23', '2020-10-01/2020-10-01']

In [12]:
#for a list of times

items_list = []
for time_stamp in times_of_interest:   #goes through times in list above
    # Query the catalog
    search = catalog.search(
        collections=["landsat-c2-l2", "landsat-c1-l2"],
        bbox=bbox_of_interest,
        datetime=time_stamp,
        # #  Uncomment the two lines below to sort the image collection based on 'cloud_cover'
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
    )

    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    items_list.append(items)

Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items


In [17]:
items_list[0]


# Downloading scenes + adding temperature data
landsat visits a location every 16 days, 
this cell goes through all the scenes found above, downloads them, and adds temperature_degC calculated from lwir / lwir11

In [33]:
# These values are retrieved from Page 12 in this handbook: 
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf
lwir_scale = 0.00341802
lwir_offset = 149.0



for items in items_list:
    for selected_item in items:
        try:
            ds = odc.stac.stac_load(
                [selected_item], bbox=bbox_of_interest
            ).isel(time=0)
            if 'lwir11' in ds: #got from plot_downloaded_scns
                temperature = ds["lwir11"].astype("float")
                temperature *= lwir_scale
                temperature += lwir_offset
                celsius = temperature - 273.15
                ds['temperature_degC'] = celsius
            if 'lwir' in ds:
                temperature = ds["lwir"].astype("float")
                temperature *= lwir_scale
                temperature += lwir_offset
                celsius = temperature - 273.15
                ds['temperature_degC'] = celsius
            #do temp calcs + add
            fn = f"{str(ds.time.values)}.nc" 
            print(fn)
            ds.to_netcdf(fn)
        except Exception as e: 
            print(f"Failed on {selected_item} with error: {e}")

2020-07-05T18:31:52.785482000.nc
2020-07-13T19:01:17.887145000.nc
2020-07-21T18:30:54.112057000.nc
2020-07-29T19:01:22.439132000.nc
2020-08-06T18:29:54.122643000.nc
2020-08-14T19:01:26.426532000.nc
2020-08-22T18:28:52.743266000.nc
2020-08-30T19:01:34.768005000.nc
2020-09-07T18:27:50.030860000.nc
2020-09-15T19:01:40.966120000.nc
2020-09-23T18:26:46.234346000.nc
2020-10-01T19:01:44.969542000.nc


AttributeError: 'ItemCollection' object has no attribute 'assets'